In [2]:
# Wide display
from IPython.core.display import display, HTML
display(HTML("<style>#notebook-container { margin-left:-14px; width:calc(100% + 27px) !important; }</style>"))

In [3]:
import json, os, re, requests, subprocess

if not os.path.exists('python-utils'):
    subprocess.check_output('git clone https://github.com/CMU-CREATE-Lab/python-utils.git', shell=True)

def exec_ipynb(filename_or_url):
    nb = (requests.get(filename_or_url).json() if re.match(r'https?:', filename_or_url) else json.load(open(filename_or_url)))
    if(nb['nbformat'] >= 4):
        src = [''.join(cell['source']) for cell in nb['cells'] if cell['cell_type'] == 'code']
    else:
        src = [''.join(cell['input']) for cell in nb['worksheets'][0]['cells'] if cell['cell_type'] == 'code']
    exec('\n'.join(src), globals())

exec_ipynb('python-utils/utils.ipynb')
notebook_wide_display()

In [4]:
import bz2, html
Stat.set_service('Purpleair download realtime')

In [ ]:
while True:
    while True:
        url = 'https://www.purpleair.com/json'
        dest_path = datetime.datetime.utcnow().strftime('mirror/%Y%m%d/%H%M%Sutc.json.bz2')
        response = requests.get(url, timeout=120)
        if response.status_code == 200:
            try:
                js = response.json()
                devices = js['results']
                break
            except Exception as ex:
                Stat.warning('Exception %s parsing result from %s' % (ex, url))
        else:
            Stat.warning('Response code %d from %s: %s' % (response.status_code, url, html.escape(response.text)))
        time.sleep(20)
    
    os.makedirs(os.path.dirname(dest_path), exist_ok=True)
    tmp_path = '%s-%d.tmp' % (dest_path, os.getpid())
    bz2.open(tmp_path, 'wb').write(response.content)
    os.rename(tmp_path, dest_path)
    Stat.up('Received and compressed %d devices (%.1f KB) to %s (%.1f KB)' % (len(devices), len(response.content)/1e3, dest_path, os.path.getsize(dest_path)/1e3),
            valid_for_secs=5*60)
    sleep_until_next_period(150)